In [2]:
import pandas as pd
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [254]:
try:
    # XXX this still contains stations with very few datapoints for prec_mm
    df2 = pd.read_csv("prepared_data.csv")
except:
    # read in the just data we're interested in 
    df = pd.read_csv("data.csv", usecols=['date', 'lat', 'lon', 'station', 'prec_mm'])
    # we're interested in precipitation; replace "Tr" (trace) with arbitrarily small figure
    df['prec_mm'] = pd.to_numeric(df['prec_mm'].str.replace("Tr", "0.01"))
    df = df.fillna(0)
    # There are summary rows in the data; remove those
    df2 = df[(~pd.isnull(df.lat)) & (df.station != 'Summary')]
    # Allow us to average over years
    df2['month-day'] = df2['date'].str.replace(r".*-(\d+)-(\d+)", r"\1-\2")
    # Skip leap year Feb 29ths for convenience
    df2 = df2[df2['month-day'] != '02-29']
    # Create a categorical variable: did it rain at all?
    df2['rained'] = df2.prec_mm.apply(lambda x: 1 if x > 0 else 0)
    # Prune stations with not-so-many data points
    counts = df2.groupby('station').count()
    total_rained = df2.groupby('station').sum()
    stations_with_data = df2.merge(
        counts[counts.date > 0.85*len(counts)], how='inner', on='station', suffixes=['', '_y'])
    # remove stations where there's never been any rain - presumably they don't measure it
    stations_with_data = stations_with_data.merge(
        total_rained[total_rained.rained > 500], how='inner', on='station', suffixes=['', '_y'])
    # Average across all years of data
    stations_with_data = stations_with_data.groupby(['month-day', 'station']).mean()
    stations_with_data = stations_with_data[['lat','lon','prec_mm','rained']]
    df2 = stations_with_data.reset_index()
    df2.write_csv("prepared_data.csv")

In [228]:
counts.head()

,date,lat,lon,prec_mm,month-day
station,,,,,
Aberdaron,6095,6095,6095,5845,6095
Aberdeen / Dyce,6837,6837,6837,6627,6837
Aberhosan,753,753,753,5,753
Aberporth,6847,6847,6847,6602,6847
Aboyne,6007,6007,6007,5660,6007


In [19]:
df2.head()

,Unnamed: 0,month-day,station,lat,lon,prec_mm,rained
0,0,01-01,Aberdaron,52.783,-4.733,2.562500,0.937500
1,1,01-01,Aberdeen / Dyce,57.200,-2.217,3.256111,0.777778
2,2,01-01,Aberporth,52.133,-4.567,3.089444,0.833333
3,3,01-01,Aboyne,57.083,-2.833,1.255333,0.866667
4,4,01-01,Albemarle,55.017,-1.867,3.844286,0.928571


In [255]:
import itertools
def p_no_rain(p_rains, no_more_than=0):
    """Calculate the probability of it raining no more than N days,
    over a series of probabilities that it rains at all.
    
    This is very inefficient, there's certainly a better way of 
    building the permutations...
    """
    final_p = 0
    # The best case outcome
    desired_outcomes = ['no_rain'] * len(p_rains)
    permutations = set((tuple(desired_outcomes),))
    # Plus all the others, up to the worst case outcome
    for n in range(0, no_more_than):
        desired_outcomes[n] = 'rain'
        for p in itertools.permutations(desired_outcomes, len(p_rains)):
            permutations.add(p)
    for permutation in permutations:
        this_p = 1
        for i, outcome in enumerate(permutation):
            if outcome == 'no_rain':
                outcome_p = 1 - p_rains.iloc[i]
            else:
                outcome_p = p_rains.iloc[i]
            this_p = this_p * outcome_p
        final_p += this_p
    return final_p

In [256]:
import datetime
from datetime import date, timedelta

def is_start_day(val, requested_day, year): 
    days = {
    'monday': 0,
    'tuesday': 1,
    'wednesday': 2,
    'thursday': 3,
    'friday': 4,
    'saturday': 5,
    'sunday': 6}
    day = val.isoweekday()
    return day == days[requested_day]


def nearest_station_name(df, location):
    from shapely.ops import nearest_points
    gdf = df.groupby('station').max()[['lat', 'lon']]
    # Compute vonoroi regions for each point
    gdf['coordinates'] = list(zip(gdf.lon, gdf.lat))
    gdf['coordinates'] = gdf['coordinates'].apply(Point)
    gdf = geopandas.GeoDataFrame(gdf, geometry='coordinates')
    pts = gdf.geometry.unary_union
    return gdf[gdf.geometry == nearest_points(location, pts)[1]].index[0]


def compute_locations(df, starting_day, days_holiday, max_days_rain_acceptable):
    date_parts = [int(x) for x in starting_day.split("-")]
    
    starting_date = date(*date_parts)
    ending_day = (starting_date + timedelta(days=days_holiday-1)).strftime("%m-%d")
    print("Checking dates", starting_date.strftime("%m-%d"), ending_day)
    df2 = df[(df['month-day'] >= starting_date.strftime("%m-%d")) & (df['month-day'] <= ending_day)]
    asd = df2.groupby('station')['rained'].apply(p_no_rain, no_more_than=max_days_rain_acceptable)
    df = pd.DataFrame(asd.sort_values(ascending=False))
    df.columns = ['p_good_weather']
    return df


def compute_dates(df, year, starting_day, days_holiday, max_days_rain_acceptable):
    df = df.copy()
    df['date'] = pd.to_datetime(df['month-day'] + '-' + str(year))
    df['is_start_day'] = df.date.apply(is_start_day, requested_day=starting_day, year=year)
    vals = []
    for row in df[df.is_start_day].iterrows():
        i = df.index.get_loc(row[0])   
        candidates = df.iloc[i:i+days_holiday]
        vals.append({'date': row[1]['date'], 'p':p_no_rain(candidates.rained, no_more_than=max_days_rain_acceptable)})
    return pd.DataFrame(vals).sort_values('p', ascending=False)

In [257]:
# Best location for a given date
starting_day = "2019-07-12"
days_holiday = 3
max_days_rain_acceptable = 0

compute_locations(df2, starting_day, days_holiday, max_days_rain_acceptable).head(20)

Checking dates 07-12 07-14


,p_good_weather
station,
Jersey Airport,0.847645
Redhill,0.400000
Fylingdales,0.398017
Saint Helier,0.331330
Saint Catherine's...,0.282257
Leeds Weather Cen...,0.281250
Rothamsted,0.277778
Sella Ness,0.257701
Great Malvern,0.246914


In [267]:
# Best date for a given location
starting_day = 'saturday'
days_holiday = 2
max_days_rain_acceptable = 0
year = 2019


stroud = Point(-2.2407643,51.7422478)
llanthony = Point(-3.1069677, 51.944618)
station = nearest_station_name(df2, llanthony)
print(station)
region = df2[df2.station == station]
asd = compute_dates(region, year, starting_day, days_holiday, max_days_rain_acceptable)
asd.head(10)

Hereford/Credenhill


,date,p
22,2019-06-07,0.335180
34,2019-08-30,0.305556
28,2019-07-19,0.299169
21,2019-05-31,0.274238
15,2019-04-19,0.274238
35,2019-09-06,0.250000
10,2019-03-15,0.250000
17,2019-05-03,0.224377
13,2019-04-05,0.224377
37,2019-09-20,0.222222


In [269]:
station = nearest_station_name(df2, stroud)
region = df2[df2.station == station]
asd = compute_dates(region, year, starting_day, days_holiday, max_days_rain_acceptable)
asd.head(10)

,date,p
37,2019-09-20,0.339506
32,2019-08-16,0.308642
13,2019-04-05,0.274238
24,2019-06-21,0.249307
20,2019-05-24,0.249307
38,2019-09-27,0.246914
14,2019-04-12,0.224377
34,2019-08-30,0.221453
31,2019-08-09,0.216049
29,2019-07-26,0.213296


In [270]:
asd[asd.date == '2019-06-07']

,date,p
22,2019-06-07,0.155125


# Working out which have too few data points

In [221]:
df3 = pd.read_csv("data.csv", usecols=['date', 'lat', 'lon', 'station', 'prec_mm'])
df3['prec_mm'] = pd.to_numeric(df3['prec_mm'].str.replace("Tr", "0.01"))

In [226]:
df4 = df3[df3.station == 'Jersey Airport']
import numpy as np
len(df4[np.isnan(df4.prec_mm)])/len(df4)

0.8576014006419609

In [208]:
df2.groupby('station')['rained'].describe().sort_values('50%')
 

,count,mean,std,min,25%,50%,75%,max
station,,,,,,,,
Jersey Airport,365.0,0.097324,0.050361,0.000000,0.052632,0.105263,0.157895,0.263158
Sella Ness,365.0,0.303635,0.096840,0.000000,0.272727,0.300000,0.363636,0.500000
Fylingdales,365.0,0.313789,0.090372,0.105263,0.263158,0.315789,0.368421,0.526316
Redhill,365.0,0.372146,0.200797,0.000000,0.200000,0.333333,0.500000,1.000000
Saint Helier,365.0,0.328680,0.108603,0.000000,0.250000,0.333333,0.416667,0.545455
"London, St James ...",365.0,0.460236,0.176715,0.000000,0.333333,0.444444,0.555556,0.888889
Saint Catherine's...,365.0,0.466408,0.134983,0.157895,0.368421,0.473684,0.578947,0.842105
Shoreham Airport,365.0,0.499648,0.139080,0.157895,0.388889,0.473684,0.578947,0.842105
Scampton,365.0,0.500219,0.136484,0.166667,0.411765,0.500000,0.588235,0.882353


In [210]:
df2[df2.station == 'Redhill']

,Unnamed: 0,month-day,station,lat,lon,prec_mm,rained
114,114,01-01,Redhill,51.217,-0.133,2.700000,0.666667
272,272,01-02,Redhill,51.217,-0.133,1.966667,0.500000
430,430,01-03,Redhill,51.217,-0.133,0.366667,0.500000
588,588,01-04,Redhill,51.217,-0.133,2.333333,0.500000
746,746,01-05,Redhill,51.217,-0.133,1.533333,0.500000
904,904,01-06,Redhill,51.217,-0.133,0.280000,0.400000
1062,1062,01-07,Redhill,51.217,-0.133,0.366667,0.666667
1220,1220,01-08,Redhill,51.217,-0.133,1.100000,0.500000
1378,1378,01-09,Redhill,51.217,-0.133,0.533333,0.500000
1536,1536,01-10,Redhill,51.217,-0.133,1.333333,0.500000
